In [3]:
#Import basics
import time
import numpy as np
import json
import random

length_limit= 8 #List Watched Movies
num_cand= 19 #List Candidate Movies
random_seed = 4321

In [2]:
#Install LLM related libraries
!pip install trl peft accelerate bitsandbytes

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
#Download Movielens 100k dataset
!wget https://github.com/marcopoli/LLaMAntino-3-ANITA/raw/main/use_examples/recsys/ml_100k.json

--2024-05-09 12:36:15--  https://github.com/AGI-Edgerunners/LLM-Next-Item-Rec/raw/main/ml_100k.json
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Next-Item-Rec/main/ml_100k.json [following]
--2024-05-09 12:36:15--  https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Next-Item-Rec/main/ml_100k.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764666 (747K) [text/plain]
Saving to: ‘ml_100k.json’

ml_100k.json        100%[===================>] 746.74K  --.-KB/s    in 0.04s   

2024-05-09 12:36:15 (17.0 MB/s) - ‘ml_100k.json’ saved [764666/764666]



In [4]:
#Read Sequences of Movies from files
def read_json(file):
    with open(file) as f:
        return json.load(f)

def write_json(data, file):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

data_ml_100k = read_json("ml_100k.json")


In [5]:
#Exctract users and associated movies from file
u_item_dict = {}
u_item_p = 0
for elem in data_ml_100k:
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in u_item_dict:
            u_item_dict[movie] = u_item_p
            u_item_p +=1
u_item_len = len(u_item_dict)

user_list = []
for i, elem in  enumerate(data_ml_100k):
    item_hot_list = [0 for ii in range(u_item_len)]
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        item_pos = u_item_dict[movie]
        item_hot_list[item_pos] = 1
    user_list.append(item_hot_list)

user_matrix = np.array(user_list)
user_matrix_sim = np.dot(user_matrix, user_matrix.transpose())
print(user_matrix)
print(user_matrix_sim)

[[1 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[45  1  2 ...  4  4  4]
 [ 1 49  0 ...  1 11  6]
 [ 2  0 50 ...  3  1  1]
 ...
 [ 4  1  3 ... 50  5  0]
 [ 4 11  1 ...  5 50  8]
 [ 4  6  1 ...  0  8 20]]


In [6]:
### Extract Candidate list from user sequences
def sort_uf_items(target_seq, us, num_u, num_i):
    candidate_movies_dict = {}
    sorted_us = sorted(list(enumerate(us)), key=lambda x: x[-1], reverse=True)[:num_u]
    dvd = sum([e[-1] for e in sorted_us])
    for us_i, us_v in sorted_us:
        us_w = us_v * 1.0/dvd

        us_elem = data_ml_100k[us_i]
        us_seq_list = us_elem[0].split(' | ')

        for us_m in us_seq_list:
            if us_m not in target_seq:
                if us_m not in candidate_movies_dict:
                    candidate_movies_dict[us_m] = 0.
                candidate_movies_dict[us_m]+=us_w

    candidate_pairs = list(sorted(candidate_movies_dict.items(), key=lambda x:x[-1], reverse=True))
    candidate_items = [e[0] for e in candidate_pairs][:num_i]
    return candidate_items

In [7]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)


base_model = "m-polignano-uniba/LLaMAntino-3-ANITA_test"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
num_u= 15 #num similar users to consider for extracting candidate list
total_i = num_cand
total = 0
cand_ids = []
id_list =list(range(0,len(data_ml_100k)))

#Create the sequences to evaluate
for i in id_list[:1000]:
    elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')

    candidate_items = sort_uf_items(seq_list, user_matrix_sim[i], num_u=num_u, num_i=total_i)
    print(candidate_items)

    if elem[-1] in candidate_items:
        cand_ids.append(i)
    else:
        pass

    total +=1


['Star Wars', 'Fargo', 'The Birdcage', 'The Godfather', 'Toy Story', 'Men in Black', 'Jerry Maguire', 'Raiders of the Lost Ark', 'The Lost World: Jurassic Park', 'Sabrina', 'Eraser', 'Star Trek: First Contact', 'Air Force One', 'Pulp Fiction', 'Twelve Monkeys', 'Blade Runner', 'Reservoir Dogs', 'Terminator 2: Judgment Day', 'Seven (Se7en)']
['Boogie Nights', 'Apt Pupil', 'Titanic', 'Tomorrow Never Dies', "Dante's Peak", 'Kiss the Girls', 'Scream', "The Devil's Advocate", 'Chasing Amy', 'The English Patient', 'Air Force One', 'Amistad', "Eve's Bayou", 'Desperate Measures', 'Murder at 1600', 'Contact', 'Starship Troopers', 'Volcano', 'Evita']
['Naked Gun 33 1/3: The Final Insult', 'Mrs. Doubtfire', 'Dumb & Dumber', 'Billy Madison', 'Coneheads', 'Ace Ventura: When Nature Calls', 'Hot Shots! Part Deux', 'Pretty Woman', 'Airheads', 'The Flintstones', 'Casper', 'The Parent Trap', 'Money Train', 'Angels in the Outfield', "Pete's Dragon", 'Virtuosity', 'Cool Runnings', 'Son in Law', 'Waterworl

In [9]:
import transformers
pipe = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # langchain expects the full text
    task='text-generation',
    max_new_tokens=512, # max number of tokens to generate in the output
    do_sample=True,
    temperature=0.1
)

In [10]:
results_data = []
count = 0
total = 0
for i in cand_ids[:10]:
    elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')[::-1]

    candidate_items = sort_uf_items(seq_list, user_matrix_sim[i], num_u=num_u, num_i=total_i)
    random.shuffle(candidate_items)

    tmp_1_s= "<CandidateSet>: {}.\n\n<WatchedMovies>: {}.\n\n"
    tmp_1_s= tmp_1_s.format(', '.join(candidate_items), ', '.join(seq_list[-length_limit:]))

    tmp_3_u = "Can you recommend 10 movies from the <CandidateSet> similar to the <WatchedMovies>?"
    messages_3 = [
        {"role": "user", "content": tmp_1_s+tmp_3_u},
    ]
    #prompt = tokenizer.apply_chat_template(messages_3, tokenize=False, add_generation_prompt=True, fast=False)

    sequences = pipe(messages_3)
    for seq in sequences:
       predictions = seq['generated_text']
    hit_=0
    if elem[1] in predictions:
        count += 1
        hit_ = 1
    else:
        pass
    total +=1


    print (f"GT: {elem[1]}")
    print(f"CandidateSet: {', '.join(candidate_items)}")
    print (f"predictions: {predictions}")

    result_json = {"PID": i,
                   "GT": elem[1],
                   "Predictions": predictions,
                   'Hit': hit_,
                   'Count': count,
                   'Current_total':total,
                   'Hit@10':count*1.0/total}
    results_data.append(result_json)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


GT: Under Siege 2: Dark Territory
CandidateSet: Star Trek IV: The Voyage Home, Star Trek: The Wrath of Khan, Money Train, Clear and Present Danger, Terminal Velocity, Natural Born Killers, The Crow, Young Guns, Batman Forever, The Shadow, Independence Day (ID4), Under Siege 2: Dark Territory, Stargate, Highlander, Waterworld, Die Hard 2, Days of Thunder, GoldenEye, First Knight
predictions: A fun challenge!

After analyzing the <WatchedMovies> and the <CandidateSet>, I've identified some patterns and genres that might be of interest to the person who watched those movies. Here are my recommendations of 10 movies from the <CandidateSet> that might be similar in tone, genre, or style to the <WatchedMovies>:

1. Batman Forever (Action, Comedy) - Similar to Beverly Hills Cop III, an action-comedy with a lighthearted tone.
2. The Crow (Action, Fantasy) - Like Highlander, a fantasy-action film with a strong protagonist.
3. Young Guns (Western, Action) - Similar to City Slickers II: The Legen

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


GT: Speed
CandidateSet: Batman Returns, Star Trek VI: The Undiscovered Country, Under Siege, Conan the Barbarian, Batman Forever, Money Train, Star Trek III: The Search for Spock, The Three Musketeers, Natural Born Killers, First Knight, The Shadow, Full Metal Jacket, GoldenEye, Batman, Stargate, The Crow, True Lies, Under Siege 2: Dark Territory, Speed
predictions: Based on the genres and styles of the movies in the <WatchedMovies> set, I'll recommend 10 movies from the <CandidateSet> that might be of interest:

1. Batman (from the <CandidateSet>) - an action-packed superhero movie, similar to the Batman franchise in the <CandidateSet>
2. Star Trek III: The Search for Spock (from the <CandidateSet>) - a science fiction film, similar to Starship Troopers in the <WatchedMovies>
3. The Crow (from the <CandidateSet>) - a dark, gothic superhero movie, similar to the tone of The Crow in the <WatchedMovies>
4. First Knight (from the <CandidateSet>) - a medieval, action-packed fantasy film, s

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

GT: Independence Day (ID4)
CandidateSet: Beavis and Butt-head Do America, My Best Friend's Wedding, Tin Cup, Bound, Sabrina, Time to Kill, A, Rumble in the Bronx, The Devil's Own, Boot, Das, The Truth About Cats & Dogs, Trainspotting, Independence Day (ID4), Mission: Impossible, Broken Arrow, Welcome to the Dollhouse, Jackie Chan's First Strike, Matilda, Willy Wonka and the Chocolate Factory, Face/Off
predictions: Based on the movies in the <WatchedMovies> set, I'll recommend 10 movies from the <CandidateSet> that share similar genres, tones, or styles. Here are my picks:

1. Mission: Impossible (Action/Thriller, similar to Mission: Impossible in the <WatchedMovies>)
2. Face/Off (Action, similar to Face/Off in the <CandidateSet>)
3. Broken Arrow (Action, similar to Broken Arrow in the <CandidateSet>)
4. Sabrina (Romantic Comedy, similar to My Best Friend's Wedding in the <CandidateSet>)
5. Welcome to the Dollhouse (Dark Comedy, similar to Welcome to the Dollhouse in the <CandidateSet>)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


GT: The Rock
CandidateSet: The First Wives Club, The River Wild, Fargo, Executive Decision, The Nutty Professor, The Truth About Cats & Dogs, The People vs. Larry Flynt, The Hunchback of Notre Dame, The Spitfire Grill, Eraser, Jerry Maguire, Grumpier Old Men, Tin Cup, Absolute Power, Down Periscope, Men in Black, The Rock, My Best Friend's Wedding, Twister
predictions: Based on the genres and directors of the movies in the <WatchedMovies> set, I'll recommend 10 movies from the <CandidateSet> that might be similar:

1. Fargo (from the <CandidateSet>) - a dark comedy-crime thriller, similar to The Long Kiss Goodnight (from the <WatchedMovies>)
2. The River Wild (from the <CandidateSet>) - an action-thriller, similar to Daylight (from the <WatchedMovies>)
3. Executive Decision (from the <CandidateSet>) - an action-thriller, similar to Broken Arrow (from the <WatchedMovies>)
4. Jerry Maguire (from the <CandidateSet>) - a romantic comedy, similar to My Best Friend's Wedding (from the <Watch

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


GT: L.A. Confidential
CandidateSet: Good Will Hunting, Raiders of the Lost Ark, The Rainmaker, My Best Friend's Wedding, Jerry Maguire, Seven Years in Tibet, L.A. Confidential, The Long Kiss Goodnight, The Bridge on the River Kwai, G.I. Jane, Chasing Amy, The People vs. Larry Flynt, Schindler's List, The Full Monty, Gandhi, The Silence of the Lambs, Star Wars, Alien, The Game
predictions: Based on the movies in the <WatchedMovies> set, I'll recommend 10 movies from the <CandidateSet> that share similar themes, genres, or awards, to the taste of the person who has watched those movies. Here are my recommendations:

1. The Silence of the Lambs (from <CandidateSet>) - a psychological thriller, similar to The Deer Hunter and The Silence of the Lambs in the <WatchedMovies> set.
2. Gandhi (from <CandidateSet>) - a biographical drama, similar to Gandhi in the <WatchedMovies> set, and also an Academy Award winner.
3. Schindler's List (from <CandidateSet>) - a historical drama, similar to Schin

In [11]:
file_dir = f"./results_multi_prompting_len{length_limit}_numcand_{total_i}_seed4321.json"
write_json(results_data, file_dir)